In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import glob
from datetime import datetime, timedelta
import json

import glob
import json
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import numpy as np

from bokeh.layouts import row, column
from bokeh.models import CustomJS, Slider
from bokeh.plotting import figure, output_file, show, ColumnDataSource, save
from iot.core import get_bitalino_df, get_glasses_df, get_tablet_df



In [2]:
glob.glob('experiments/*')

['experiments\\exp1.txt',
 'experiments\\exp2.txt',
 'experiments\\exp3.txt',
 'experiments\\exp4.txt',
 'experiments\\exp5.txt',
 'experiments\\exp6.txt']

In [3]:
experiment_file = 'experiments/exp3.txt'
experiment_name = experiment_file.split('/')[-1].split('.')[0]

In [4]:
with open(experiment_file) as f:
    lines = f.read().split('\n')[:3]

In [5]:
bitalino_path, glasses_path, tablet_path = lines

In [6]:
dic_df = {}
if bitalino_path:
    dic_df['bitalino'], _ = get_bitalino_df(bitalino_path, N = 100000)
dic_df['glasses'], _ = get_glasses_df(glasses_path, N = 10000)
if tablet_path:
    dic_df['tablet'], _ = get_tablet_df(tablet_path, N = 10000)

In [7]:
js_pattern = """
    const data = source.data;
    {}
    for (var i = 0; i < data['xs'][1].length; i++) {
        data['xs'][1][i] = data['xs2'][1][i]+phase.value+phase2.value;
    }
    source.change.emit();
"""

In [8]:
base_name  = 'bitalino'
for sync_name in ['glasses', 'tablet'][1:]:
    

    source_dict = dict(
        xs=[dic_df[base_name]['exp_time'].values, dic_df[sync_name]['exp_time'].values],
        ys=[dic_df[base_name]['sync_col'].values, dic_df[sync_name]['sync_col'].values ],
        line_color = ['red','blue']*2
        )

    source_dict['xs2'] = source_dict['xs']

    source_dict['legend'] = [base_name, sync_name]*2
    source_dict['line_width'] = [2]*2+[0]*2



    p = figure(width=1500, height=500) 
    source = ColumnDataSource(source_dict)
    delta = dic_df[base_name]['sysdatetime'].iloc[0] - dic_df[sync_name]['sysdatetime'].iloc[0]
    delta = delta.total_seconds()


    phase_slider = Slider(start=delta-100, end=delta+100, value=delta, step=.1, title="Phase")
    phase_slider2 = Slider(start=-2, end=2, value=0, step=.001, title="Phase2")


    callback = CustomJS(args=dict(source=source,phase=phase_slider, phase2=phase_slider2),
                        code=js_pattern.replace('{}','phase2.value=0;'))
    callback2 = CustomJS(args=dict(source=source,phase=phase_slider, phase2=phase_slider2),
                        code=js_pattern.replace('{}',''))


    phase_slider.js_on_change('value', callback)
    phase_slider2.js_on_change('value', callback2)

    p.multi_line(xs="xs", ys="ys",source=source,
                    line_color='line_color',
                 legend='legend',
                    line_width='line_width'
                )

    layout = column(p, phase_slider, phase_slider2)
    output_file(f'C:/Users/artem/PycharmProjects/iot/output/{experiment_name}_{sync_name}.html')
    save(layout)